In [1]:
!pip install lenskit
!pip install -U numba

import lenskit.datasets as ds
import pandas as pd

!git clone https://github.com/crash-course-ai/lab4-recommender-systems.git

data = ds.MovieLens('lab4-recommender-systems/')

print("Successfully installed dataset.")

  Using cached numba-0.51.0-cp38-cp38-macosx_10_14_x86_64.whl (2.2 MB)
  Using cached llvmlite-0.34.0-cp38-cp38-macosx_10_9_x86_64.whl (18.4 MB)
ERROR: lenskit 0.10.1 has requirement numba<0.51,>=0.45, but you'll have numba 0.51.0 which is incompatible.
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.33.0+1.g022ab0f
ERROR: Cannot uninstall 'llvmlite'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
fatal: destination path 'lab4-recommender-systems' already exists and is not an empty directory.


In [2]:
rows_to_show = 10   #Try changing this number to see more rows of data
data.ratings.head(rows_to_show) 

,user,item,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


In [3]:
joined_data = data.ratings.join(data.movies['genres'], on='item')
joined_data = joined_data.join(data.movies['title'], on='item')
joined_data.head(rows_to_show)

,user,item,rating,timestamp,genres,title
0,1,1,4.0,964982703,Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995)
1,1,3,4.0,964981247,Comedy|Romance,Grumpier Old Men (1995)
2,1,6,4.0,964982224,Action|Crime|Thriller,Heat (1995)
3,1,47,5.0,964983815,Mystery|Thriller,Seven (a.k.a. Se7en) (1995)
4,1,50,5.0,964982931,Crime|Mystery|Thriller,"Usual Suspects, The (1995)"
5,1,70,3.0,964982400,Action|Comedy|Horror|Thriller,From Dusk Till Dawn (1996)
6,1,101,5.0,964980868,Adventure|Comedy|Crime|Romance,Bottle Rocket (1996)
7,1,110,4.0,964982176,Action|Drama|War,Braveheart (1995)
8,1,151,5.0,964984041,Action|Drama|Romance|War,Rob Roy (1995)
9,1,157,5.0,964984100,Comedy|War,Canadian Bacon (1995)


In [4]:
average_ratings = (data.ratings).groupby(['item']).mean()
sorted_avg_ratings = average_ratings.sort_values(by="rating", ascending=False)
joined_data = sorted_avg_ratings.join(data.movies['genres'], on='item')
joined_data = joined_data.join(data.movies['title'], on='item')
joined_data = joined_data[joined_data.columns[1:]]

print("RECOMMENDED FOR ANYBODY:")
joined_data.head(rows_to_show)

RECOMMENDED FOR ANYBODY:


,rating,timestamp,genres,title
item,,,,
88448,5.0,1.315438e+09,Comedy|Drama,Paper Birds (Pájaros de papel) (2010)
100556,5.0,1.456151e+09,Documentary,"Act of Killing, The (2012)"
143031,5.0,1.520409e+09,Comedy|Drama|Romance,Jump In! (2007)
143511,5.0,1.526207e+09,Documentary,Human (2015)
143559,5.0,1.520410e+09,Comedy|Crime|Fantasy,L.A. Slasher (2015)
6201,5.0,1.100120e+09,Drama|Romance,Lady Jane (1986)
102217,5.0,1.443200e+09,Comedy,Bill Hicks: Revelations (1993)
102084,5.0,1.493422e+09,Action|Animation|Fantasy,Justice League: Doom (2012)
6192,5.0,1.063275e+09,Romance,Open Hearts (Elsker dig for evigt) (2002)


In [5]:
average_ratings = (data.ratings).groupby('item') \
       .agg(count=('user', 'size'), rating=('rating', 'mean')) \
       .reset_index()

sorted_avg_ratings = average_ratings.sort_values(by="rating", ascending=False)
joined_data = sorted_avg_ratings.join(data.movies['genres'], on='item')
joined_data = joined_data.join(data.movies['title'], on='item')
joined_data = joined_data[joined_data.columns[1:]]


print("RECOMMENDED FOR ANYBODY:")
joined_data.head(rows_to_show)

#but this the recommendation of one single user

RECOMMENDED FOR ANYBODY:


,count,rating,genres,title
7638,1,5.0,Comedy|Drama,Paper Birds (Pájaros de papel) (2010)
8089,1,5.0,Documentary,"Act of Killing, The (2012)"
9065,1,5.0,Comedy|Drama|Romance,Jump In! (2007)
9076,1,5.0,Documentary,Human (2015)
9078,1,5.0,Comedy|Crime|Fantasy,L.A. Slasher (2015)
4245,1,5.0,Drama|Romance,Lady Jane (1986)
8136,1,5.0,Comedy,Bill Hicks: Revelations (1993)
8130,1,5.0,Action|Animation|Fantasy,Justice League: Doom (2012)
4240,1,5.0,Romance,Open Hearts (Elsker dig for evigt) (2002)
9104,1,5.0,Comedy,Formula of Love (1984)


In [6]:
minimum_to_include = 20 #<-- You can try changing this minimum to include movies rated by fewer or more people

average_ratings = (data.ratings).groupby(['item']).mean()
rating_counts = (data.ratings).groupby(['item']).count()
average_ratings = average_ratings.loc[rating_counts['rating'] > minimum_to_include]
sorted_avg_ratings = average_ratings.sort_values(by="rating", ascending=False)
joined_data = sorted_avg_ratings.join(data.movies['genres'], on='item')
joined_data = joined_data.join(data.movies['title'], on='item')
joined_data = joined_data[joined_data.columns[3:]]

print("RECOMMENDED FOR ANYBODY:")
joined_data.head(rows_to_show)

RECOMMENDED FOR ANYBODY:


,genres,title
item,,
318,Crime|Drama,"Shawshank Redemption, The (1994)"
922,Drama|Film-Noir|Romance,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)
898,Comedy|Drama|Romance,"Philadelphia Story, The (1940)"
475,Drama,In the Name of the Father (1993)
1204,Adventure|Drama|War,Lawrence of Arabia (1962)
246,Documentary,Hoop Dreams (1994)
858,Crime|Drama,"Godfather, The (1972)"
1235,Comedy|Drama|Romance,Harold and Maude (1971)
168252,Action|Sci-Fi,Logan (2017)


In [7]:
average_ratings = (data.ratings).groupby(['item']).mean()
rating_counts = (data.ratings).groupby(['item']).count()
average_ratings = average_ratings.loc[rating_counts['rating'] > minimum_to_include]
average_ratings = average_ratings.join(data.movies['genres'], on='item')
average_ratings = average_ratings.loc[average_ratings['genres'].str.contains('Action')]

sorted_avg_ratings = average_ratings.sort_values(by="rating", ascending=False)
joined_data = sorted_avg_ratings.join(data.movies['title'], on='item')
joined_data = joined_data[joined_data.columns[3:]]
print("RECOMMENDED FOR AN ACTION MOVIE FAN:")
joined_data.head(rows_to_show)

RECOMMENDED FOR AN ACTION MOVIE FAN:


,genres,title
item,,
168252,Action|Sci-Fi,Logan (2017)
2959,Action|Crime|Drama|Thriller,Fight Club (1999)
58559,Action|Crime|Drama|IMAX,"Dark Knight, The (2008)"
1197,Action|Adventure|Comedy|Fantasy|Romance,"Princess Bride, The (1987)"
260,Action|Adventure|Sci-Fi,Star Wars: Episode IV - A New Hope (1977)
3275,Action|Crime|Drama|Thriller,"Boondock Saints, The (2000)"
1208,Action|Drama|War,Apocalypse Now (1979)
1196,Action|Adventure|Sci-Fi,Star Wars: Episode V - The Empire Strikes Back...
1233,Action|Drama|War,"Boot, Das (Boat, The) (1981)"


In [8]:
average_ratings = (data.ratings).groupby(['item']).mean()
rating_counts = (data.ratings).groupby(['item']).count()
average_ratings = average_ratings.loc[rating_counts['rating'] > minimum_to_include]
average_ratings = average_ratings.join(data.movies['genres'], on='item')
average_ratings = average_ratings.loc[average_ratings['genres'].str.contains('Romance')]

sorted_avg_ratings = average_ratings.sort_values(by="rating", ascending=False)
joined_data = sorted_avg_ratings.join(data.movies['title'], on='item')
joined_data = joined_data[joined_data.columns[3:]]
print("RECOMMENDED FOR A ROMANCE MOVIE FAN:")
joined_data.head(rows_to_show)

RECOMMENDED FOR A ROMANCE MOVIE FAN:


,genres,title
item,,
922,Drama|Film-Noir|Romance,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)
898,Comedy|Drama|Romance,"Philadelphia Story, The (1940)"
1235,Comedy|Drama|Romance,Harold and Maude (1971)
912,Drama|Romance,Casablanca (1942)
1197,Action|Adventure|Comedy|Fantasy|Romance,"Princess Bride, The (1987)"
933,Crime|Mystery|Romance|Thriller,To Catch a Thief (1955)
908,Action|Adventure|Mystery|Romance|Thriller,North by Northwest (1959)
4973,Comedy|Romance,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ..."
356,Comedy|Drama|Romance|War,Forrest Gump (1994)
